<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colB5BERT_fine_tuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Fine tuning colBERT with Big Five dataset

* Notebook inspirado no buscador denso de Leandro Carísio Fernandes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers


In [3]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull


Cloning into 'colB5BERT'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 254 (delta 43), reused 18 (delta 18), pack-reused 196
Receiving objects: 100% (254/254), 32.69 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (158/158), done.
/content/colB5BERT
Already up to date.


### Libraries

In [4]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst

import os
import pandas as pd
import numpy as np
import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm

import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer
from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

import torch
from transformers import AutoModel

from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW
from tqdm.auto import tqdm

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 26 20:38:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [40]:
df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test.tsv', sep='\t')
df_train = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_train.tsv', sep='\t')

len(df_test)
len(df_train)

# Sample data to test the pipeline

df_test = df_test.sample(frac=0.08)
df_train = df_train.sample(frac=0.08)
len(df_test)

df_test = df_test.sample(n = 1000)
df_train = df_train.sample(n = 1000)

,coditem,domain,pole,item_pt_text,id,id_divisao,texto_dividido,postive_ex
1388235,sv2.299,E,1,Ficar animado quando as coisas dão certo.,382,2,fizeram ! ! ! : https : www . $NAME$ . com wat...,1
809284,sv2.613,O,0,Não me interesso sobre problemas que nunca ter...,869,11,Diante da tua grandeza O que se vê o coração s...,0
1450361,sv2.331,E,1,Não tenho medo de falar as coisas que penso.,696,21,"obrigado de nada . """""""" As novinhas e mais vel...",0
3677822,STA_OPT10,N,0,Sempre acho que coisas ruins irão acontecer,1102,6,""" Pois bem ! Diga mais nada ! $LAUGH$ aquário ...",0
3426769,STA_TRU04,A,1,Eu acredito que meus amigos nunca irão me trair,641,12,. . Amor da minha vida Daqui até a eternidade ...,1
...,...,...,...,...,...,...,...,...
853154,sv2.708,O,1,Aprender sobre novas culturas.,269,5,"$NAME$ , cancerianos e eu temos problemas . Qu...",1
2493180,STA_TOL10,O,0,Evito pessoas que têm uma aparência e comporta...,19,5,estar feliz Hoje é dia de ir trabalhar com a m...,0
2608014,STA_PER01,C,1,Continuo trabalhando em uma tarefa até terminar,850,6,$NAME$ precisa se importar mais com a copa do ...,0
515593,bfi2_76,O,1,Gosto de atividades artísticas,1162,2,"recordar , pantera cor de $NAME$ ! Bom dia ! B...",0


In [29]:
queries_train = df_train['item_pt_text'].tolist()
docs_train = df_train['texto_dividido'].tolist()
positive_ex_train = df_train['postive_ex'].tolist()


queries_val = df_test['item_pt_text'].tolist()
docs_val = df_test['texto_dividido'].tolist()
positive_ex_val = df_test['postive_ex'].tolist()

type(queries_val)


len(queries_train)

1000

## Fine-tuning dos encoders

Define os datasets e dataloaders:

In [14]:
max_length = 512
batch_size = 50
epochs = 4
lr = 2e-5


In [72]:
# Definição do Dataset
class Dataset(data.Dataset):
    # Recebe dois vetores de textos e um vetor de labels
    def __init__(self, tokenizer, textos, labels, max_seq_length = max_length):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.textos = textos
        self.labels = labels
        self.cache = {}

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        self.cache[str(idx)] = self.cache.get(str(idx),
                   (self.tokenizer(self.textos[idx],
                                  padding=True,
                                  truncation=True,
                                  max_length=self.max_seq_length
                                  ),
                    self.labels[idx])
                   )
        return self.cache[str(idx)]


# Definição do Dataset
class Dataset(data.Dataset):
    # Recebe dois vetores de textos e um vetor de labels
    def __init__(self, tokenizer, textos, labels, max_seq_length = max_length):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.textos = textos
        self.labels = labels
        self.cache = {}

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        self.cache[str(idx)] = self.cache.get(str(idx),
                   (self.tokenizer(self.textos[idx],
                                  padding=True,
                                  truncation=True,
                                  max_length=self.max_seq_length
                                  ))
                   )
        return self.cache[str(idx)]



In [74]:
# Create datasets
bert_weights_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(bert_weights_name)

dataset_queries_train = Dataset(tokenizer, queries_train, positive_ex_train)
dataset_docs_train = Dataset(tokenizer, docs_train, positive_ex_train)

dataset_queries_val = Dataset(tokenizer, queries_val, positive_ex_val)
dataset_docs_val = Dataset(tokenizer, docs_val, positive_ex_val)


# Dataloaders para os datasets

#collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
#def collate_fn(batch):
#    #print('Dentro de collate_fn')
#    #print(BatchEncoding(tokenizer.pad(batch, return_tensors='pt')))
#    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

# collate function that also handles the labels
def collate_fn(batch):
    inputs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    return BatchEncoding(tokenizer.pad(inputs, return_tensors='pt')), torch.tensor(labels)


# Create dataloaders
dataloader_queries_train = DataLoader(dataset_queries_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_train = DataLoader(dataset_docs_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

dataloader_queries_val = DataLoader(dataset_queries_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_val = DataLoader(dataset_docs_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)



# Adapt your DataLoader object
#class PositiveExDataLoader:
#    def __init__(self, dataloader, positive_ex):
#        self.dataloader = dataloader
#        self.positive_ex = positive_ex
#
#    def __iter__(self):
#        for batch, pos_ex in zip(self.dataloader, self.positive_ex):
#            yield batch, pos_ex
#    def __len__(self):
#        return len(self.dataloader)
## Wrap your original dataloaders
#dataloader_queries_train = PositiveExDataLoader(dataloader_queries_train, positive_ex_train)
#dataloader_docs_train = PositiveExDataLoader(dataloader_docs_train, positive_ex_train)
#
#dataloader_queries_val = PositiveExDataLoader(dataloader_queries_val, positive_ex_val)
#dataloader_docs_val = PositiveExDataLoader(dataloader_docs_val, positive_ex_val)


Carrega os modelos:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Se tiver que treinar os modelos, abre
model_query = AutoModel.from_pretrained(bert_weights_name).to(device)
model_doc = AutoModel.from_pretrained(bert_weights_name).to(device)

Define função pro cálculo da loss (modifiquei essa função para trabalhar com um vetor indicando quais pares são relevantes e quais não são:

In [75]:
epsilon = 1e-8
# Essa função já considera o resultado via batchs:
def compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs, positive_ex):
    outputs_queries = model_query(**tokenized_queries.to(device))
    outputs_docs    = model_doc(**tokenized_docs.to(device))

   # outputs_queries = model_query(**{k: v.to(device) for k, v in tokenized_queries.items()})
  #  outputs_docs    = model_doc(**{k: v.to(device) for k, v in tokenized_docs.items()})

  #  outputs_queries = model_query(**tokenized_queries)
  #  outputs_docs    = model_doc(**tokenized_docs)


    print(tokenized_queries.items())
    print(tokenized_docs.items())



    print(outputs_queries)
    print(outputs_docs)

    # Extrai a última camada oculta associada ao token [CLS]
    tcls_queries = outputs_queries.last_hidden_state[:, 0, :]
    tcls_docs    = outputs_docs.last_hidden_state[:, 0, :]
    print(tcls_queries)
    print(tcls_docs )


    # Normaliza os tensores
    #tcls_queries = tcls_queries / torch.norm(tcls_queries, dim=1, keepdim=True)
    #tcls_docs = tcls_docs / torch.norm(tcls_docs, dim=1, keepdim=True)

    # Agora é necessário calcular a loss. Para isso, o primeiro passo é
    # calcular a similaridade entre uma query e documento (sim(q, d))
    similaridade = torch.matmul(tcls_queries, torch.transpose(tcls_docs, 0, 1))
    print(similaridade)
    # Calcula a exponencial da similaridade
    exp_sim = torch.exp(similaridade)
    print(exp_sim)
    # Calcula a loss
    # We are now considering only the positive examples (where positive_ex is 1)
    positive_exp_sim = exp_sim * positive_ex.to(device)
    print(positive_exp_sim)
    soma_linhas = positive_exp_sim.sum(dim=1)
    diagonal = torch.diag(positive_exp_sim)
    log_loss = -1* torch.log(diagonal/(soma_linhas + epsilon))

    loss = torch.mean(log_loss)
    return loss

def compute_loss_sem_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs, positive_ex):
    with torch.no_grad():
        return compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs, positive_ex)

def compute_loss_dataloaders(model_query, dataloader_query, model_doc, dataloader_docs):
    loss = 0
    n_batches = 0
    for (batch_query, positive_ex_query), (batch_docs, positive_ex_docs)  in zip(dataloader_query, dataloader_docs):
        positive_ex_query = positive_ex_query.to(device)
        positive_ex_docs = positive_ex_docs.to(device)
        loss = loss + compute_loss_sem_gradiente(model_query, batch_query, model_doc, batch_docs, positive_ex_query)
        n_batches += 1
    return loss/n_batches

In [76]:
%%time
# Só pra medir o tempo que ele demora para calcular a loss em todo o dataset de treinamento
model_query.eval()
model_doc.eval()
print(f'Loss de treinamento: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
print(f'Loss de validação: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

IndexError: ignored

Agora treina os dois encoders simulatenamente:

In [35]:
%%time

treinar_e_salvar_modelos  = True
# Diretório onde vai salvando o modelo a cada época
dir_modelos = '/content/drive/MyDrive/colB5BERT'


if treinar_e_salvar_modelos:
  # Training loop
  optimizer_query = AdamW(model_query.parameters(), lr=lr)
  optimizer_doc = AdamW(model_doc.parameters(), lr=lr)

  num_training_steps = epochs * len(dataloader_queries_train)
  num_warmup_steps = int(num_training_steps * 0.1)

  # get_linear_schedule_with_warmup get_cosine_with_hard_restarts_schedule_with_warmup
  scheduler_query = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_query, num_warmup_steps, num_training_steps)
  scheduler_doc = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_doc, num_warmup_steps, num_training_steps)

  for epoch in tqdm(range(epochs), desc='Epochs'):
      model_query.train()
      model_doc.train()

      train_losses = []
      for (batch_query, positive_ex_query), (batch_docs, positive_ex_docs) in tqdm(list(zip(dataloader_queries_train, dataloader_docs_train)), mininterval=0.5, desc='Train', disable=False):
        optimizer_query.zero_grad()
        optimizer_doc.zero_grad()

        # Ensure positive_ex_query and positive_ex_docs are tensors and on the same device as your models
        batch_query = batch_query.to(device)
        batch_docs = batch_docs.to(device)
        positive_ex_query = torch.tensor(positive_ex_query).to(device)
        positive_ex_docs = torch.tensor(positive_ex_docs).to(device)

        loss = compute_loss_com_gradiente(model_query, batch_query, model_doc, batch_docs, positive_ex_query)
        loss.backward()

        optimizer_query.step()
        optimizer_doc.step()

        scheduler_query.step()
        scheduler_doc.step()


      model_query.save_pretrained(f'{dir_modelos}{epoch}/query/')
      model_doc.save_pretrained(f'{dir_modelos}{epoch}/doc/')

      model_query.eval()
      model_doc.eval()

      print(f'Loss de treinamento {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
      print(f'Loss de validação {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Train:   0%|          | 0/20 [00:00<?, ?it/s]

<timed exec>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
<timed exec>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Loss de treinamento 0: nan
Loss de validação 0: nan


Train:   0%|          | 0/20 [00:00<?, ?it/s]

Loss de treinamento 1: nan
Loss de validação 1: nan


Train:   0%|          | 0/20 [00:00<?, ?it/s]

Loss de treinamento 2: nan


KeyboardInterrupt: ignored